In [1]:
from langchain_ollama import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain

In [8]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
    Essa é a uma conversa amigavel entre um humano e uma IA.
    Responda em português.
                                               
    Conversa atual:
    {history}                                              
    Human:{input}
    AI:
""")

### Cadeia de Conversação

In [9]:
chat = chat = ChatOllama(model="llama3:8b")

memory = ConversationBufferMemory()
chain = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True, 
    prompt=prompt_template
)

In [10]:
chain.predict(input="Olá")



> Entering new ConversationChain chain...
Prompt after formatting:

    Essa é a uma conversa amigavel entre um humano e uma IA.
    Responda em português.

    Conversa atual:
                                                  
    Human:Olá
    AI:


> Finished chain.


'AI: Olá! Como vai?'

### LLM Chain

In [11]:
from langchain_ollama import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain

chat = chat = ChatOllama(model="llama3:8b")

In [12]:
prompt = PromptTemplate.from_template("""
    Escolha o melhor nome para mim sobre uma emprsa que desenvolve soluções em {produto}
    Em português
""")

chain = LLMChain(llm=chat, prompt=prompt)
produto =  "Carros hibridos"
chain.run(produto)

C:\Users\frpbo\AppData\Local\Temp\ipykernel_6428\1662884878.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat, prompt=prompt)
C:\Users\frpbo\AppData\Local\Temp\ipykernel_6428\1662884878.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(produto)


'Que desafio legal!\n\nAqui estão algumas sugestões de nomes para a sua empresa:\n\n1. **Hybrida Energia**: combina a ideia de carro híbrido com o conceito de energia renovável.\n2. **Ecocarros**: uma abreviação que conjuga "eco" (ecologia) e "carros" para transmitir a mensagem de sustentabilidade.\n3. **EcoMotion**: nome que combina "eco" (ecologia) com "motion" (movimento) para criar um termo que evoca a ideia de carros híbridos movendo-se em direção a uma melhor sustentabilidade.\n4. **GreenDrive**: nome que destaca a ênfase na sustentabilidade e no uso eficiente da energia, com "green" (verde) e "drive" (dirigir).\n5. **Carro Verde Sustentável** (ou CVS): um nome simples e direto que comunica a proposta de sustentabilidade.\n6. **Inovação Mobilidade**: um nome que destaca a inovação e a mobilidade, transmitindo a ideia de soluções inovadoras para o setor de carros híbridos.\n7. **EcoRides**: uma abreviação que combina "eco" (ecologia) com "rides" (viagens) para criar um termo diver

### Simple Sequential Chain

In [21]:
from langchain_ollama import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.chains.sequential import SimpleSequentialChain, SequentialChain

chat = chat = ChatOllama(model="llama3:8b")

In [18]:
prompt = PromptTemplate.from_template("""
    Escolha o melhor nome para mim sobre uma emprsa que desenvolve soluções em {produto}
    Em português
""")
chain_nome = LLMChain(llm=chat, prompt=prompt)


prompt_description = PromptTemplate.from_template("""
    Sobre a empresa com nome{nome_empresa}
    Informe uma descrição de até 20 palavras
    Em português
""")

chain_descripiton = LLMChain(llm=chat, prompt=prompt_description)


In [19]:

chain = SimpleSequentialChain(
    chains=[chain_nome,chain_descripiton],
    verbose=True
)
produto =  "Carros hibridos"
chain.run(input=produto)



> Entering new SimpleSequentialChain chain...
Um desafio divertido!

Aqui estão algumas sugestões de nomes para sua empresa que desenvolve soluções em carros híbridos:

1. **Hybridia**: uma combinação das palavras "híbrido" e "inovação", sugerindo a busca por novas soluções.
2. **GreenDrive**: um nome que combina a preocupação com o meio ambiente (green) com a ideia de dirigir (drive), enfatizando a eficiência energética dos carros híbridos.
3. **EcoMobil**: uma combinação das palavras "ecológico" e "mobilidade", ressaltando a preocupação com o ambiente e a mobilidade urbana.
4. **Hybridium**: um nome que combina as palavras "híbrido" e "universo", sugerindo a capacidade da empresa de criar soluções universais para carros híbridos.
5. **Energia Verde**: um nome que destaca a energia renovável e a sustentabilidade, associando à ideia de carros híbridos.
6. **MobiHib**: uma combinação das palavras "mobilidade" e "híbrido", ressaltando a capacidade da empresa de criar soluções para carr

'Entendi!\n\nPara mim, o nome mais atraente é **GreenDrive**. Ele combina perfeitamente a preocupação com o meio ambiente e a mobilidade urbana, enfatizando a eficiência energética dos carros híbridos.\n\n"Empreendimento que desenvolve soluções inovadoras para carros híbridos, priorizando sustentabilidade e redução de impacto ambiental."'

### Senquential Chain

In [28]:
prompt = PromptTemplate.from_template("""
    Qual um bom nome de emprsa que desenvolve soluções em {produto}
    Em português
""")
chain_nome = LLMChain(llm=chat, prompt=prompt, output_key="nome_empresa")


prompt_description = PromptTemplate.from_template("""
    Sobre a empresa com nome{nome_empresa}
    Informe uma descrição de até 20 palavras
    Em português
""")

chain_descripiton = LLMChain(llm=chat, prompt=prompt_description, output_key="descricao_empresa")


prompt_traducao = PromptTemplate.from_template("""
    Crie um nome em espanhol para empresa de nome {nome_empresa}
    que possui a seguinte descrição {descricao_empresa}
""")

chain_traducao = LLMChain(llm=chat, prompt=prompt_traducao, output_key="nome_espanhol")


In [29]:
chain = SequentialChain(
    chains=[chain_nome,chain_descripiton,chain_traducao],
    input_variables=["produto"],
    output_variables=["nome_empresa","descricao_empresa","nome_espanhol"],
    verbose=True
)
produto =  "Carros hibridos"
chain.invoke(input=produto)



> Entering new SequentialChain chain...

> Finished chain.


{'produto': 'Carros hibridos',
 'nome_empresa': 'Aqui estão algumas sugestões de nomes para uma empresa que desenvolve soluções em carros híbridos:\n\n1. **Hybrid Motors**: Um nome direto e simples que destaca a especialização da empresa nos motores híbridos.\n2. **EcoMobilis**: Uma combinação de "eco" (ecológico) e "mobilis" (em movimento), sugerindo uma empresa que se concentra em soluções para veículos mais sustentáveis.\n3. **Green Drive Technologies**: Um nome que destaca a ênfase na sustentabilidade e tecnologia, sugerindo uma empresa que desenvolve inovações para melhorar a eficiência dos carros híbridos.\n4. **Hibridia**: Uma combinação de "híbrido" e o sufixo "-ia", sugerindo uma empresa que se especializa em soluções híbridas para veículos.\n5. **Renova Energia**: Um nome que destaca a ênfase na renovação da energia e no uso sostenível dos recursos, sugerindo uma empresa que desenvolve soluções para reduzir o impacto ambiental dos carros.\n6. **Carro Verde**: Um nome simples 